In [8]:

from __future__ import division
import psycopg2
import sys, nltk, re, string, heapq, gensim
from nltk.corpus import stopwords
from psycopg2 import Error
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation
import math

from collections   import Counter
from prettytable   import PrettyTable
from textblob      import TextBlob
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.cluster import KMeans
from gensim import corpora, models
import unidecode

from es_lemmatizer import lemmatize
import spacy

import pandas as pd
import pandas.io.sql as sqlio
from optimus import Optimus
from translate import Translator

# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from google.cloud import storage

#nltk.download('stopwords')
sw_en = stopwords.words('english')
sw_es = stopwords.words('spanish')
sw_en.extend(['bbva'])
sw_es.extend(['bbva'])

def crearConexion():
    
    connection = psycopg2.connect(user = "m.soto.montesinos",
                                      password = "1234",
                                      host = "127.0.0.1",
                                      port = "5432",
                                      database = "twitterBBVA")

    return connection

def obtenerTweets():

    try:
        
        connection = crearConexion()
        query = "select status_id, text, lang, sentimiento from training_set_bueno"
        
        tweets = sqlio.read_sql_query(query, connection)
        
    except (Exception, psycopg2.DatabaseError) as error :
    
        print ("Error obteniendo los datos", error)
    
    finally:
        
        if (connection):
            
            connection.close()
            print("PostgreSQL connection is closed")

    return tweets

def guardar_datos_db(tweet_id, sentiment):

    try:
        
        connection = crearConexion()
        cursor = connection.cursor()
        
        query = """
                UPDATE training_set_bueno SET sentimiento = '%f' where status_id = '%s';
                """ % (float(sentiment), tweet_id)
        
        cursor.execute(query)
        connection.commit()
        
    except (Exception, psycopg2.DatabaseError) as error :
    
        print ("Error obteniendo los datos", error)
    
    finally:
        
        if (connection):
            
            connection.close()
            cursor.close()
            print("PostgreSQL connection is closed")

def tokenizar_tweet(tweet, stop_words):
    
    tokens = TweetTokenizer(strip_handles = True, reduce_len = True).tokenize(tweet)
    tokens = [w.lower() for w in tokens]
    #tokens = [w.lower() for w in tokens if w not in stop_words and len(w) > 2 and w.isalpha()]
    #tokens = [unidecode.unidecode(w) for w in tokens]
    
    return " ".join(tokens)

def obtencion_sentimiento(tweet, client, lng_txt):
    
    sentimiento = 0
    stop_words = sw_es
        
    if lng_txt == "en":
            
        stop_words = sw_en
            
    tweet = tokenizar_tweet(tweet, stop_words)
        
    if len(tweet) > 0:
        
        try:
                
            document = types.Document(content=tweet, type=enums.Document.Type.PLAIN_TEXT)
            doc_sentiment = client.analyze_sentiment(document=document).document_sentiment
            sentimiento = doc_sentiment.score
        
        except:
    
            print ("Google Api error")
        
    return sentimiento
    

def main():
    
    tweets = obtenerTweets()
    client = language.LanguageServiceClient()
    
    for index, row in tweets.iterrows():
    
        tweet = (row['text'])
        lng_txt = (row['lang'])
        tweet_id = (row['status_id'])
        
        sentimiento = obtencion_sentimiento(tweet, client, lng_txt)
        
        guardar_datos_db(tweet_id, sentimiento)
        
        print(str(index) + ": " + tweet + " : " + str(sentimiento) + "\n")
    
if __name__ == '__main__':
    main()





PostgreSQL connection is closed
PostgreSQL connection is closed
0: Todo mundo habla del diferencial de tasas como algo que contribuye a la estabilidad del tipo de cambio. Sobre ese tema platiqué con @alexismilo  de HSBC y con Carlos Serrano de BBVA en el #PerasyManzanas de la semana. Por Spotify, iTunes, Google podcasts y demás. 🍐🍎🍐🍎 https://t.co/wu82QiDir4 : 0.30000001192092896

PostgreSQL connection is closed
1: Today in 2011, BBVA Compass Stadium breaks ground east of Downtown. @HoustonDynamo @BBVACompassStdm @HoustonDash : 0.4000000059604645

PostgreSQL connection is closed
2: Villarejo atribuyó a «patriotas» del CNI su éxito en el primer proyecto para el BBVA https://t.co/TpYM557aoa : 0.30000001192092896

PostgreSQL connection is closed
3: Mi mamá debe ser la única persona en este país al menos que dice Bilbao Vizcaya en vez de BBVA, yo no lo entiendos : 0.0

PostgreSQL connection is closed
4: Liked on YouTube: Resumen | Puebla vs América | Liga BBVA MX - CL 2020  - Jornada 1 http

PostgreSQL connection is closed
41: Comunicación organizacional en clave sonora: el caso de Blink, los podcasts de BBVA // Organizational communication in sound format: The case of Blink, the BBVA's podcasts https://t.co/tLcHYep7w3 via @EprintsELIS : 0.5

PostgreSQL connection is closed
42: Anticorrupción acusa a Francisco González de contratar “de forma opaca” a Villarejo para el BBVA https://t.co/K7KM0ZbV9a vía @elpais_economia : -0.10000000149011612

PostgreSQL connection is closed
43: @jzuazola @migsebastiang @bbva @elpais_economia @lodares @BernaldoDQuiros @MiguelGTormo @AmadeoPetitbo Negar que Miguel sea un buen economista es absurdo, máxime sin conocerle.
Yo le he visto actuar durante años en el bbv..... : -0.10000000149011612

PostgreSQL connection is closed
44: @elpais_economia Es increíble hasta donde llega la ambición de las personas.
No tenia suficiente este señor con lo ganado en la operación de #Argentaria y #BBVA y su gran sueldo como presidente?
#anticorrupcion #Villare


PostgreSQL connection is closed
77: @AlfredooGar @SantanderMx @BBVA_Mex Abusado JAJAJAJA me da como entre risa y coraje ssss al tiro gracias por el dato pa. : 0.10000000149011612

PostgreSQL connection is closed
78: El ex jefe de Seguridad del BBVA Corrochano atribuye a Francisco González la contratación de Villarejo. Asegura que fue una “orden directa” suya y que le dio el “mandato” de  que solo podía despachar ese tema con él y con dos directivos más.
https://t.co/ADFdEeRpuu : 0.0

PostgreSQL connection is closed
79: #Exclusiva El testimonio clave en el 'caso BBVA': el ex directivo Béjar implica a Francisco González en la contratación de Villarejo
https://t.co/VYvhGOVSQs : 0.30000001192092896

PostgreSQL connection is closed
80: @bbva_argentina necesito comunicarme con ustedes al privado : 0.10000000149011612

PostgreSQL connection is closed
81: Economía/Motor.- González usó de forma privada a Villarejo para adquirir fincas: el "proyecto Rick" que pagó el #BBVA https://t.co/r7TEMxrs


PostgreSQL connection is closed
116: @faustodelarrosa Y uno sufriendo con el af20. Disney... Y bbva... : 0.20000000298023224

PostgreSQL connection is closed
117: Pinches cajeros de @bbva #bancomer como si me sobrara tiempo para venir a reclamar por el dinero que se tragó la practicaja : -0.6000000238418579

PostgreSQL connection is closed
118: No es posible que ni siquiera acudiendo a la sucursal puedan responder al robo de mi dinero por parte de @bbva #bancomer #Zacatecas #México se supone que son confiables sus practicajas. #5deFebrero : -0.4000000059604645

PostgreSQL connection is closed
119: @R8309 @bancosantander @Citibanamex @bbva @HSBC_MX @BancoAfirme @GFBanorte_mx @conducef @AsocBancosMx @BancoAzteca @bancomundialmx @bancomext @Banorte_mx @BanorteEscucha @Profeco Uno bueno???? Ninguno, pues... NTM : -0.10000000149011612

PostgreSQL connection is closed
120: @RedActivismo @LadyGrupoModelo @alangonz @FueraMancera @Rodpac @Barbariux_ @RinBosque @VozCdMx @MiniXochitl @JaviEsquiv


PostgreSQL connection is closed
155: Villarejo accedió a una “ingente cantidad” de datos policiales en sus trabajos para el BBVA https://t.co/sWspi2g819 https://t.co/EEr13OaXeB : 0.0

PostgreSQL connection is closed
156: Budweiser | Typical American | 2020 Super Bowl Commercial https://t.co/36x2QMPwwW via @YouTube Need Cash Quick? Get Up To $10,000 Now. Email dated 4/24/17.
livery (REPEATED TWEET)
*reloading-"BBVA, Deutsche Bank, Erste Bank, GAZPROMBANK, Renaissance..." : 0.10000000149011612

PostgreSQL connection is closed
157: Testimonio clave en el 'caso BBVA': el ex directivo Béjar implica a Francisco González en la contratación de Villarejo https://t.co/qeIrgEDLVr : 0.20000000298023224

PostgreSQL connection is closed
158: @BBVA_Colombia su campaña uga uga desatiende al adulto mayor. Pésimo servicio en la oficina de belmira en Bogota : -0.30000001192092896

PostgreSQL connection is closed
159: Ahora en la Torre BBVA: Ceremonia de entrega de Becas BBVA para Chavos que Inspiran 201

...


PostgreSQL connection is closed
10814: RESULTADOS DE LOS PRONOSTICOS.

GRACIAS A :

@beesportmexico

@netbet_mx

@vmgcases

LIGA BBVA MX.

#LIGAMX #MEXICO #PRONOSTICOS https://t.co/bycizK2Ohr : 0.20000000298023224

PostgreSQL connection is closed
10815: Hace cuatro años Torre BBVA, era inaugurada, en esa noche, la torre nos regaló la primera iluminación de sus fachadas y sorprendió a los capitalinos con su interesante pirotecnia.  #torrebbva #bbva #inauguracion… https://t.co/oNkLPfK0oN : 0.30000001192092896

PostgreSQL connection is closed
10816: Estadio Corregidora, los Gallos Blancos perdieron en casa ante su similar de América., en el partido correspondiente a la jornada 5 de la Liga BBVA https://t.co/0rftHfuoK1 : -0.30000001192092896

PostgreSQL connection is closed
10817: Estadio Corregidora, los Gallos Blancos perdieron en casa ante su similar de América., en el partido correspondiente a la jornada 5 de la Liga BBVA. https://t.co/Acd6tvn93S : -0.20000000298023224

PostgreSQL con

PostgreSQL connection is closed
10857: @taylorskere @bbva_argentina @VisaArgentina @JeffBezos Eu yo estaba a 5 metros y cuando lo vi hacer eso pensé “mauri Swiftie!!!11” y después me reía porque lo hacía re mal jalsks : -0.10000000149011612

PostgreSQL connection is closed
10858: @taylorskere @bbva_argentina @VisaArgentina Pero es re random, porque recién compré una taza y un buzo de Grey’s y me dejó 😤😤😤 @JeffBezos hace algo amigo : -0.6000000238418579

PostgreSQL connection is closed
10859: @taylorskere @bbva_argentina @VisaArgentina @JeffBezos nuuu la estoy usando ahora jaksjs 💕 https://t.co/Cke5JHYYau : 0.5

PostgreSQL connection is closed
10860: @EmiSwiftie13 @bbva_argentina @VisaArgentina @JeffBezos Nuuuu, reemplazaste mi taza😔 : -0.4000000059604645

PostgreSQL connection is closed
10861: @taylorskere @bbva_argentina @VisaArgentina @JeffBezos Che igual cuando asumió vos tenías 11 AÑOS FUA SOPHIA : -0.10000000149011612

PostgreSQL connection is closed
10862: @taylorskere @bbva_arge


PostgreSQL connection is closed
10904: @faaviolasanchez Es de buena suerte, cómprate el boleto del avión presidencial, así nos llevas a todos al bbva : 0.4000000059604645

PostgreSQL connection is closed
10905: POR QUÉ BLOQUEASTE MI TARJETA?😢 qué daño te hiceeee? 😭 @BBVA_Mex : -0.20000000298023224

PostgreSQL connection is closed
10906: Marine ecologists Daniel Pauly, Carlos Duarte and Terenece Hughes receives 12th Frontiers of Knowledge Award in Ecology and Conservation Biology from BBVA Foundation. Triple congratulations!!! https://t.co/2P3MDifPz9 : 0.30000001192092896

PostgreSQL connection is closed
10907: @billy_1247 Ojalá siempre jueguen en el barrial no se merecen el bbva la neta : 0.20000000298023224

PostgreSQL connection is closed
10908: ¿Todxs amamos BBVA? Todxs amamos BBVA. 💙 : 0.4000000059604645

PostgreSQL connection is closed
10909: Estimado @BBVA_Mex donde puedo reportar que tu cajero de bombas y cafetales número 3736 no me dio dinero pero si me cobró en mi cuenta el r


PostgreSQL connection is closed
10943: @migsebastiang @bbva @elpais_economia Al que usted no puede ver porque le le antoje el puesto en el Servicio de Estudios y se lo dio a otro. Visto lo visto en ese asunto una cargado de motivos. Mejor cállese, que es usted parte. : -0.30000001192092896

PostgreSQL connection is closed
10944: León consigue triunfo en el Barrial

En acción de la sexta jornada de la LIGA BBVA MX Femenil, Rayadas recibió al Club León...
https://t.co/YPhYsIRQM6 https://t.co/TXVFGW4cmt : 0.30000001192092896

PostgreSQL connection is closed
10945: Igualito que cuando expulsaron a Lautaro, le enseña la amarilla y ya tenia la mano en el bolsillo para sacar la roja, el unico consuelo que tengo es que los arbitros de la BBVA son iguales o peores que los de la Serie A https://t.co/bYPAKwxqhG : -0.699999988079071

PostgreSQL connection is closed
10946: Eso de los puntos @bbva es un fraude, no valen nada!! : -0.8999999761581421

PostgreSQL connection is closed
10947: @israelbar


PostgreSQL connection is closed
10982: Salí de mi trabajo y me topé a los laguneros del @ClubSantos partiendo al BBVA que estuvieron hospedados en el hotel cercano a mi edificio https://t.co/el81i5k1uc : 0.20000000298023224

PostgreSQL connection is closed
10983: @edukaito15 @lopezobrador_ @GobiernoMX @cceoficialmx @bbva @carlosslim @TelcelEmpresas @ServicioTelcel @hectorslimseade @arturoelias @IFT_MX @SCT_mx @Profeco @TelevisaTVmx @eazcarraga @RicardoBSalinas @SalinasBenjamin @Azteca @mileniotv @PartidoMorenaMx @AP X eso demostremos #Mexico quien manda, les aseguro q ahora si será Satisfacción Total de el Cliente y no d sus absurdas acumulaciones q lo único q han creado es una sociedad decadente sin respeto ya a punto de explotar, donde todos perderemos, los ricos 1o atte #ElCacasObrador : -0.8999999761581421

PostgreSQL connection is closed
10984: @draveropdeg @Ginalaranita Ay le$ hablan empresario$ vorace$ @carlosslim @hectorslimseade @ServicioTelcel @arturoelias Carlo$ $alazar @bb